In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_stones"
cohort = "GSE73680"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_stones"
in_cohort_dir = "../../input/GEO/Kidney_stones/GSE73680"

# Output paths
out_data_file = "../../output/preprocess/Kidney_stones/GSE73680.csv"
out_gene_data_file = "../../output/preprocess/Kidney_stones/gene_data/GSE73680.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_stones/clinical_data/GSE73680.csv"
json_path = "../../output/preprocess/Kidney_stones/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profile of Randall's Plaque tissue"
!Series_summary	"Randall’s plaque (RP) is the origin of renal calcification on which idiopathic calcium oxalate (CaOx) kidney stones develop. To establish genomic pathogenesis of RP, we performed the microarray analysis for comparing the gene expressions among renal papillary RP and normal tissue of 23 CaOx and 6 calcium phosphate (CaP) stone formers, and normal papillary tissue of 7 control patients. Compare to normal papillary tissue, RP tissue contained up-regulation of lipocalin 2, interleukin 11, prostaglandin-endoperoxide synthase 1, glutathione peroxidase 3, and monocyte to macrophage differentiation, whereas down-regulation of solute carrier family 12 member 1  and sodium leak channel non selective (either > 2.0- or 0.5-fold, p <0.01). The network and toxicity analysis showed these genes had association with activated mitogen-activated protein kinase, Akt/ phosphatidylinositol 3-kinase pa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Check gene expression data availability
# Based on background information, this dataset contains gene expression data from microarray analysis
is_gene_available = True

# 2. Variable availability and data type conversion
# 2.1 Data Availability
# Looking at the Sample Characteristics Dictionary
# trait_row: We can infer the trait (kidney stones) from the lesion or tissue type information
trait_row = 1  # 'lesion' seems to indicate presence of Randall's Plaque
age_row = None  # Age information is not available in the sample characteristics
gender_row = 0  # Gender information is available

# 2.2 Data Type Conversion functions
def convert_trait(value):
    """Convert trait information to binary value (0: no kidney stones, 1: kidney stones)"""
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert based on lesion type
    if 'Plaque' in value:
        return 1  # Randall's Plaque present (kidney stone)
    elif 'Normal' in value:
        return 0  # Normal mucosa (no kidney stone)
    else:
        return None

def convert_age(value):
    """Convert age information (not used as age is not available)"""
    return None

def convert_gender(value):
    """Convert gender information to binary (0: female, 1: male)"""
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert gender to binary
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    # We need to use the clinical_data that should have been loaded previously
    # and not try to recreate it from the sample characteristics dictionary
    
    # Check if clinical data exists from previous steps
    try:
        # Assuming clinical_data is available from previous steps
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        print("Preview of extracted clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    except NameError:
        print("Clinical data not found. Make sure it was loaded in a previous step.")
    except Exception as e:
        print(f"Error processing clinical data: {e}")


Preview of extracted clinical features:
{'GSM1900673': [0.0, 0.0], 'GSM1900674': [0.0, 1.0], 'GSM1900675': [0.0, 0.0], 'GSM1900676': [1.0, 0.0], 'GSM1900677': [0.0, 1.0], 'GSM1900678': [1.0, 1.0], 'GSM1900679': [0.0, 1.0], 'GSM1900680': [1.0, 1.0], 'GSM1900681': [1.0, 1.0], 'GSM1900682': [0.0, 1.0], 'GSM1900683': [1.0, 1.0], 'GSM1900684': [0.0, 0.0], 'GSM1900685': [0.0, 1.0], 'GSM1900686': [0.0, 0.0], 'GSM1900687': [1.0, 0.0], 'GSM1900688': [0.0, 1.0], 'GSM1900689': [1.0, 1.0], 'GSM1900690': [0.0, 1.0], 'GSM1900691': [1.0, 1.0], 'GSM1900692': [0.0, 1.0], 'GSM1900693': [1.0, 1.0], 'GSM1900694': [0.0, 1.0], 'GSM1900695': [1.0, 1.0], 'GSM1900696': [0.0, 1.0], 'GSM1900697': [1.0, 1.0], 'GSM1900698': [0.0, 1.0], 'GSM1900699': [1.0, 1.0], 'GSM1900700': [0.0, 1.0], 'GSM1900701': [1.0, 1.0], 'GSM1900702': [0.0, 0.0], 'GSM1900703': [0.0, 1.0], 'GSM1900704': [1.0, 1.0], 'GSM1900705': [0.0, 1.0], 'GSM1900706': [1.0, 1.0], 'GSM1900707': [0.0, 0.0], 'GSM1900708': [1.0, 0.0], 'GSM1900709': [1.0, 1.0

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Kidney_stones/GSE73680/GSE73680_series_matrix.txt.gz


Gene data shape: (50739, 62)
First 20 gene/probe identifiers:
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492', 'A_19_P00315493',
       'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, they appear to be Agilent microarray probe IDs
# rather than standard human gene symbols. They follow the format A_19_P00315452, etc.
# which is typical for Agilent microarray platforms. These will need to be mapped
# to human gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the gene annotation columns
print("\nAnalyzing gene symbol related columns:")
if 'GENE_SYMBOL' in gene_annotation.columns:
    print(f"Sample GENE_SYMBOL values: {gene_annotation['GENE_SYMBOL'].dropna().head(5).tolist()}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0]
print(f"\nGene data first ID: {gene_data_id_prefix}")

# Look for columns that might contain probe IDs
print("\nAnalyzing potential probe ID columns:")
if 'ID' in gene_annotation.columns:
    print(f"Sample ID values: {gene_annotation['ID'].head(5).tolist()}")
    
if 'SPOT_ID' in gene_annotation.columns:
    print(f"Sample SPOT_ID values: {gene_annotation['SPOT_ID'].head(5).tolist()}")

# Check if there's any match between gene data index and annotation IDs
print("\nChecking for overlap between gene data IDs and annotation:")
gene_data_ids = set(gene_data.index[:1000])  # Get a sample of gene data IDs
annotation_ids = set(gene_annotation['ID'].astype(str)[:1000])
overlap = gene_data_ids.intersection(annotation_ids)
print(f"Number of IDs that match between gene data and annotation 'ID' column: {len(overlap)}")
print(f"Sample overlapping IDs: {list(overlap)[:5] if overlap else 'None'}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'LOCUSLINK_ID', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing family A, member 3'], 'UNIGENE_ID': [nan, nan, 'Hs.642618', 'Hs.348522', 'Hs.360989'], 'ENSEMBL_ID': [nan, nan, 'ENST000000

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation dataframe contain gene IDs and gene symbols
# From the previous output, we can see that:
# - The ID column in gene_annotation contains probe identifiers (e.g., A_23_P117082)
# - The GENE_SYMBOL column contains gene symbols (e.g., HEBP1, KCNE4)
# - There is some overlap between gene_data index and gene_annotation ID column

# 2. Extract the gene ID and gene symbol columns to create a mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Preview of gene mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression dataframe shape after mapping: {gene_data.shape}")
print("Preview of gene expression dataframe (first 5 genes, first 5 samples):")
print(gene_data.iloc[:5, :5])

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Gene mapping dataframe shape: (46204, 2)
Preview of gene mapping dataframe:
              ID          Gene
2   A_23_P117082         HEBP1
3  A_33_P3246448         KCNE4
4  A_33_P3318220        BPIFA3
5  A_33_P3236322  LOC100129869
6  A_33_P3319925          IRG1


Gene expression dataframe shape after mapping: (20353, 62)
Preview of gene expression dataframe (first 5 genes, first 5 samples):
          GSM1900673  GSM1900674  GSM1900675  GSM1900676  GSM1900677
Gene                                                                
A1BG       -1.082109   -2.449026   -1.054509   -0.551611   -3.563693
A1BG-AS1    0.194566    0.426283    0.711320    0.702709   -0.451157
A1CF       -0.782465   -1.308139   -0.578092    0.341932   -0.704205
A2LD1      -0.567108   -0.713197    1.067966    1.425751   -1.585089
A2M         1.057854    0.512311    0.077415    1.521002   -0.386456


Gene expression data saved to: ../../output/preprocess/Kidney_stones/gene_data/GSE73680.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        # Check if trait is biased
        trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
        if trait_type == "binary":
            is_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            is_biased = judge_continuous_variable_biased(linked_data, trait)
            
        # Remove biased demographic features
        if "Age" in linked_data.columns:
            age_biased = judge_continuous_variable_biased(linked_data, 'Age')
            if age_biased:
                linked_data = linked_data.drop(columns='Age')
                
        if "Gender" in linked_data.columns:
            gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
            if gender_biased:
                linked_data = linked_data.drop(columns='Gender')
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to Randall's plaque tissue, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data from Randall's plaque tissue associated with kidney stones."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for kidney stones analysis."
    )
    
    print("Dataset is not usable for kidney stones analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (20353, 62)
Gene data shape after normalization: (19847, 62)


Normalized gene expression data saved to ../../output/preprocess/Kidney_stones/gene_data/GSE73680.csv
Original clinical data preview:
         !Sample_geo_accession  \
0  !Sample_characteristics_ch1   
1  !Sample_characteristics_ch1   
2  !Sample_characteristics_ch1   

                                          GSM1900673  \
0                                     gender: female   
1                              lesion: Normal mucosa   
2  tissue type: normal papillary tissue from cont...   

                                          GSM1900674  \
0                                       gender: male   
1                              lesion: Normal mucosa   
2  tissue type: normal papillary tissue from cont...   

                                          GSM1900675  \
0                                     gender: female   
1                              lesion: Normal mucosa   
2  tissue type: normal papillary tissue from calc...   

                                         GSM1900676  \

Linked data shape before processing: (62, 19849)
Linked data preview (first 5 rows, 5 columns):
            Kidney_stones  Gender      A1BG  A1BG-AS1      A1CF
GSM1900673            0.0     0.0 -1.082109  0.194566 -0.782465
GSM1900674            0.0     1.0 -2.449026  0.426283 -1.308139
GSM1900675            0.0     0.0 -1.054509  0.711320 -0.578092
GSM1900676            1.0     0.0 -0.551611  0.702709  0.341932
GSM1900677            0.0     1.0 -3.563693 -0.451157 -0.704205


Data shape after handling missing values: (62, 19849)
For the feature 'Kidney_stones', the least common label is '1.0' with 29 occurrences. This represents 46.77% of the dataset.
For the feature 'Gender', the least common label is '0.0' with 22 occurrences. This represents 35.48% of the dataset.


Linked data saved to ../../output/preprocess/Kidney_stones/GSE73680.csv
